# Necessary Weather Data

In this Jupyter notebook we note the necessary variables needed for PV production calculations. Moreover, we are going to list a series of possible weather databases.

## Weather databases

1. Meteonorm 8.1
2. PVGIS TMY
3. SOLCast TMY
4. Solar Anywhere TGY
5. Solargis TMY
6. NASA-SSE

## Weather variables to take into account

1. Global Horizontal Irradiance (GHI - kWh/m^2/month)
2. Horizontal Diffuse Radiation (DHI - kWh/m^2/month)
3. Temperature (celcius)
4. Wind velocity (m/s)
5. Linke turbidity (-)
6. Relative Humidity (%)

###### Side note on Linke Turbidity (TL)

Linke turbidity (TL) is really a lumped “haziness” index that accounts for all the extra extinction of sunlight beyond what a perfectly clean, dry atmosphere would do. That “extra extinction” comes mainly from two things:

Aerosols (pollution, dust, smoke, volcanic particles, etc.)
This is the “pollution” part. Urban smog or Saharan dust events push TL upward.

Water vapor absorption
Humid tropical air or summer days with high precipitable water also increase TL.

TL isn’t only a pollution indicator. A very humid but clean region (say, the Amazon) will also have a high TL even if the air is not polluted.

In [41]:
# Fetch hourly PVGIS SARAH-3 with pvlib (multi-year)
# pip install pvlib pandas pyarrow  (pyarrow optional for Parquet)

from __future__ import annotations
import datetime
import pathlib, time
import pandas as pd
import pvlib

LAT, LON = 41.9028, 12.4964
START_YEAR = 2011       
N_YEARS    = 12                
RADDATABASE = "PVGIS-SARAH3"
API_BASE = "https://re.jrc.ec.europa.eu/api/v5_3/"
OUTDIR = pathlib.Path("pvgis_sarah3_hourly"); OUTDIR.mkdir(exist_ok=True)
TILT = 21
AZIMUTH = 0

def fetch_year(year: int) -> pd.DataFrame:
    """Return hourly PVGIS data (UTC index) with pvlib-mapped column names."""
    start, end = f"{year}-01-01", f"{year}-12-31"
    df, meta = pvlib.iotools.get_pvgis_hourly(
        LAT, LON,
        start=start, end=end,
        raddatabase=RADDATABASE,
        components=True,               # get GHI/DHI/DNI etc.
        surface_tilt=TILT, 
        surface_azimuth=AZIMUTH,  # horizontal
        usehorizon=True,
        map_variables=True,
        url=API_BASE,
        outputformat="json",
    )
    return df.sort_index()


all_years = []
for i in range(N_YEARS):
    y = START_YEAR + i
    print(f"Fetching {y} ({RADDATABASE}) …")
    for attempt in range(1, 4):
        try:
            dfx = fetch_year(y)
            dfx['TL'] = pvlib.clearsky.lookup_linke_turbidity(dfx.index, LAT, LON).values
            if dfx.index.tz is None or dfx.index.tz == datetime.timezone.utc:
                dfx.index = dfx.index.tz_convert("Europe/Rome")
            all_years.append(dfx)
            break
        except Exception as e:
            if attempt == 3:
                raise
            time.sleep(1.5 * attempt)



Fetching 2011 (PVGIS-SARAH3) …
Fetching 2012 (PVGIS-SARAH3) …
Fetching 2013 (PVGIS-SARAH3) …
Fetching 2014 (PVGIS-SARAH3) …
Fetching 2015 (PVGIS-SARAH3) …
Fetching 2016 (PVGIS-SARAH3) …
Fetching 2017 (PVGIS-SARAH3) …
Fetching 2018 (PVGIS-SARAH3) …
Fetching 2019 (PVGIS-SARAH3) …
Fetching 2020 (PVGIS-SARAH3) …
Fetching 2021 (PVGIS-SARAH3) …
Fetching 2022 (PVGIS-SARAH3) …


In [40]:
all_years[0].head()

,poa_direct,poa_sky_diffuse,poa_ground_diffuse,solar_elevation,temp_air,wind_speed,Int,TL
time,,,,,,,,
2011-01-01 01:10:00+01:00,0.0,0.0,0.0,0.0,5.88,1.52,0,2.792742
2011-01-01 02:10:00+01:00,0.0,0.0,0.0,0.0,5.55,1.66,0,2.792742
2011-01-01 03:10:00+01:00,0.0,0.0,0.0,0.0,5.36,1.66,0,2.792742
2011-01-01 04:10:00+01:00,0.0,0.0,0.0,0.0,4.88,1.79,0,2.792742
2011-01-01 05:10:00+01:00,0.0,0.0,0.0,0.0,4.64,1.72,0,2.792742


# Power Output Calculation

In [ ]:
# --- CONFIG: choose whether to reuse PVGIS POA or recompute with pvlib ---
USE_PVGIS_POA = True   # A: reuse PVGIS 'poa_*' columns
# USE_PVGIS_POA = False  # B: recompute POA from GHI/DHI/DNI with Perez

import pandas as pd
import pvlib

df = pd.concat(all_years).sort_index()

df['poa_global'] = (
    df['poa_direct'] + df['poa_sky_diffuse'] + df['poa_ground_diffuse']
)

loc = pvlib.location.Location(LAT, LON, tz="Europe/Rome")

cecmods = pvlib.pvsystem.retrieve_sam('cecmod')
cecinvs = pvlib.pvsystem.retrieve_sam('cecinverter')

module = cecmods['Hanwha_HSL60P6_PA_4_250T']
inverter = cecinvs['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
temp_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']

# DC sizing (adjust to your plant):
# Let's say 100 kWdc nameplate built from 250 W modules:
P_dc_nameplate = 100_000.0  # Wdc at STC
P_mod_stc = module['STC'] if 'STC' in module else module['Impo']*module['Vmpo']
n_modules = int(round(P_dc_nameplate / P_mod_stc))
# inverter parameters are at the *inverter* level; choose a central/string inverter accordingly
# If using multiple inverters, scale by count or use pvlib's Array config for more detail.

system = pvlib.pvsystem.PVSystem(
    surface_tilt=TILT,
    surface_azimuth=AZIMUTH,
    module_parameters=module,
    inverter_parameters=inverter,
    temperature_model_parameters=temp_params,
    albedo=0.2,  # set site-appropriate albedo
)

mc = pvlib.modelchain.ModelChain(
    system, loc,
    transposition_model='perez',     # used only if we recompute POA
    aoi_model='physical',
    spectral_model='no_loss',
    losses_model='pvwatts'           # simple aggregate losses model
)

# 3) Build the weather frame expected by ModelChain
#    (index must be localized; yours already is Europe/Rome)
weather_cols = ['temp_air', 'wind_speed']
need_ghi = ['ghi', 'dni', 'dhi']
need_poa = ['poa_global','poa_direct','poa_sky_diffuse','poa_ground_diffuse']


weather = df[weather_cols + need_poa].copy()

# 4) Run the model
mc.run_model(weather)

# Results:
# mc.results.ac -> hourly AC power in W
# mc.results.dc -> dict-like with DC outputs (Pdc, Vmp, Imp, etc.)
ac = mc.results.ac.rename('P_ac_W')

# 5) Energy summaries
hourly_ac_kwh = (ac / 1000.0)  # each row is ~1h from PVGIS hourly, so Wh -> kWh
annual_kwh = hourly_ac_kwh.resample('Y').sum().rename('E_ac_kWh')
total_kwh = annual_kwh.sum()

print("Annual AC energy (kWh):")
print(annual_kwh)
print(f"\nTotal over {annual_kwh.index[0].year}–{annual_kwh.index[-1].year}: {total_kwh:,.0f} kWh")

# Optional: Performance Ratio (quick-and-dirty)
# PR = E_ac / (Plane-of-array irradiance * P_dc_nameplate / 1000) aggregated over time.
if USE_PVGIS_POA:
    poa = df['poa_global']  # W/m2
else:
    # If recomputing POA, mc.total_irrad['poa_global'] holds the computed series
    poa = mc.results.total_irrad['poa_global']

# POA irradiance (kWh/m²) per year:
poa_kwhm2_y = (poa / 1000.0).resample('Y').sum().rename('H_poa_kWh_m2')
# Plane-of-array energy at STC per kWdc ~ H_poa * (efficiencies not considered here)
# A simpler PR proxy: PR = E_ac / (H_poa * (P_dc_nameplate/1000))
pr_year = (annual_kwh / (poa_kwhm2_y * (P_dc_nameplate/1000))).rename('PR')
summary = pd.concat([annual_kwh, poa_kwhm2_y, pr_year], axis=1)
print("\nYearly summary:")
print(summary)

# If you want a single tidy DataFrame with hourly results:
result_hourly = pd.concat([ac, df.get('poa_global', poa)], axis=1)
